<a href="https://colab.research.google.com/github/Pushkar-Bhuse/Political-Influence/blob/master/SentimentAnalysis-(SVC/TF-IDF).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
# from sklearn.model_selection import train_test_split

from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report, confusion_matrix

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
downloaded = drive.CreateFile({'id':"1gJcmE0W1uFiHuhuivCjE1MA2hTZJuXNW"})   # replace the id with id of file you want to access
downloaded.GetContentFile('train.csv')        # replace the file name with your file

In [0]:
downloaded = drive.CreateFile({'id':"1y3FJe_RN9m8kBK5jthChmRez21F5gx5H"})   # replace the id with id of file you want to access
downloaded.GetContentFile('test.csv')  

In [12]:
train_data = pd.read_csv("train.csv", encoding='ISO-8859-1')
train_data.head()

,ItemID,Sentiment,SentimentText
0,1,0,is so sad for my APL frie...
1,2,0,I missed the New Moon trail...
2,3,1,omg its already 7:30 :O
3,4,0,.. Omgaga. Im sooo im gunna CRy. I'...
4,5,0,i think mi bf is cheating on me!!! ...


In [16]:
#Checking Tweets
rand_indexs = np.random.randint(1,len(train_data),10).tolist()
train_data["SentimentText"][rand_indexs]

26272    @adorableD yahhh dar, u missed the announcemen...
80564    @artistjanebush Thanks! My head is bobbin'.  I...
43973                                 @anisalovesu Right? 
83925    @CateP36  Thats awesome! I take it Twitter jus...
85632    @audaciaray @debaucheddiva I tried to text 266...
60006    @Belleslife Im spewing Poh got out she was my ...
12943    *tired* researching publishers and proof readi...
93334    @Contendo Perhaps I could be of assistance  We...
45344    @aNorthernSoul That's a nice pic. Thx for shar...
79989    @chamcircuit im going 2 try your comp but as i...
Name: SentimentText, dtype: object

In [19]:
tweets_text = train_data.SentimentText.str.cat()
emos = set(re.findall(r" ([xX:;][-']?.) ",tweets_text))
emos_count = []
for emo in emos:
    emos_count.append((tweets_text.count(emo), emo))
sorted(emos_count,reverse=True)[:10]

[(3281, ':/'),
 (2874, 'x '),
 (2626, ': '),
 (1339, 'x@'),
 (1214, 'xx'),
 (1162, 'xa'),
 (984, ';3'),
 (887, 'xp'),
 (842, 'xo'),
 (713, ';)')]

In [20]:
HAPPY_EMO = r" ([xX;:]-?[dD)]|:-?[\)]|[;:][pP]) "
SAD_EMO = r" (:'?[/|\(]) "
print("Happy emoticons:", set(re.findall(HAPPY_EMO, tweets_text)))
print("Sad emoticons:", set(re.findall(SAD_EMO, tweets_text)))

Happy emoticons: {'xD', ':D', ':d', ':p', 'x)', ';-)', ';D', 'xd', ';)', ':-D', ';p', ';P', ';-D', ';d', 'XD'}
Sad emoticons: {':|', ':/', ":'(", ':('}


In [26]:
# nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
def most_used_words(text):
    tokens = word_tokenize(text)
    frequency_dist = nltk.FreqDist(tokens)
    print("There is %d different words" % len(set(tokens)))
    return sorted(frequency_dist,key=frequency_dist.__getitem__, reverse=True)

In [28]:
most_used_words(train_data.SentimentText.str.cat())[:10]

There is 134028 different words


['@', '!', '.', 'I', ',', 'to', 'the', 'you', '?', 'a']

In [31]:
# nltk.download("stopwords")

mw = most_used_words(train_data.SentimentText.str.cat())
most_words = []
for w in mw:
    if len(most_words) == 1000:
        break
    if w in stopwords.words("english"):
        continue
    else:
        most_words.append(w)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
There is 134028 different words


In [0]:
def stem_tokenize(text):
    stemmer = SnowballStemmer("english")
    stemmer = WordNetLemmatizer()
    return [stemmer.lemmatize(token) for token in word_tokenize(text)]

def lemmatize_tokenize(text):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(token) for token in word_tokenize(text)]

In [40]:
train_data['SentimentText'] = train_data['SentimentText'].str.replace("#", "")
train_data['SentimentText'] = train_data['SentimentText'].str.replace(r"[-\.\n]", "")
# Removing HTML garbage
train_data['SentimentText'] = train_data['SentimentText'].str.replace(r"&\w+;", "")
# Removing links
train_data['SentimentText'] = train_data['SentimentText'].str.replace(r"https?://\S*", "")
# replace repeated letters with only two occurences
# heeeelllloooo => heelloo
train_data['SentimentText'] = train_data['SentimentText'].str.replace(r"(.)\1+", r"\1\1")
# mark emoticons as happy or sad
train_data['SentimentText'] = train_data['SentimentText'].str.replace(HAPPY_EMO, " happyemoticons ")
train_data['SentimentText'] = train_data['SentimentText'].str.replace(SAD_EMO, " sademoticons ")
train_data['SentimentText'] = train_data['SentimentText'].str.lower()

train_data.head(10)

,ItemID,Sentiment,SentimentText
0,1,0,is so sad for my apl friend
1,2,0,i missed the new moon trailer
2,3,1,omg its already 7:30 :o
3,4,0,omgaga im soo im gunna cry i've been at thi...
4,5,0,i think mi bf is cheating on me!! t_t
5,6,0,or i just worry too much?
6,7,1,juusst chillin!!
7,8,0,sunny again work tomorrow sademoticons tv...
8,9,1,handed in my uniform today i miss you already
9,10,1,hmm i wonder how she my number @)


In [0]:
#Handling tags, In this case we remove all the tags
train_data['SentimentText'] = train_data['SentimentText'].str.replace(r"@[a-zA-Z0-9_]* ", "")

In [0]:
sentiments = train_data['Sentiment']
tweets = train_data['SentimentText']

#using TF-IDF Vectorizer
vectorizer = TfidfVectorizer(tokenizer=lemmatize_tokenize, ngram_range=(1,2))

clf = SVC(probability=True)

grid_search_pipeline = Pipeline([
    ('vectorizer', vectorizer),
    ('SVC', clf)
])

params = [
    {
        'vectorizer__max_features': [1000, 2000, 5000, 10000, 20000, None],
        'vectorizer__ngram_range': [(1,1), (1,2)],
        'SVC__C': [0.1, 1, 10, 100, 1000],  
        'SVC__gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
        'SVC__kernel': ['rbf', 'linear', 'poly']
    },
]

grid_search = GridSearchCV(grid_search_pipeline, params, cv=5, scoring='f1')
grid_search.fit(tweets, sentiments)
print(grid_search.best_params_)

In [43]:
#nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True